# Notebook to evaluate the loss function results

This notebook is designed as an evaluation notebook.

In [1]:
%matplotlib notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import csv
import time
import os
import ants
import pandas as pd
import statsmodels.api as sm
import numpy as np
from nibabel import load as load_nii
from nibabel import Nifti1Image
from skimage.filters import threshold_otsu
from skimage.measure import label as bwlabeln
from shutil import copyfile
import torch
from torch import nn
from torch.nn import functional as F

plt.rcParams.update({'font.size': 16})

## Statistic functions

In [2]:
def get_fold_dict(csv_files, path):
    dicts = []
    keys = []
    for file in csv_files:
        csv_name = os.path.join(path, file)
        with open(csv_name) as csv_file:
            reader = csv.reader(csv_file)
            keys = next(reader)
            results_dict = {}
            for key in keys[1:-1]:
                results_dict[key] = []
            for row in reader:
                for key, data in zip(keys[1:-1], row[1:-1]):
                    results_dict[key].append(float(data))
        dicts.append(results_dict)

    final_dict = {}
    for key in keys[1:-1]:
        metrics = np.stack([metrics[key] for metrics in dicts], axis=0)
        final_dict['min_' + key] = np.min(metrics, axis=0)
        final_dict['std-_' + key] = np.mean(metrics, axis=0) - np.std(metrics, axis=0)
        final_dict['mean_' + key] = np.mean(metrics, axis=0)
        final_dict['std+_' + key] = np.mean(metrics, axis=0) + np.std(metrics, axis=0)
        final_dict['max_' + key] = np.max(metrics, axis=0)
        final_dict['all_' + key] = metrics
    return final_dict


def get_all_dicts(path):
    files = os.listdir(path)
    
    all_dicts = []
    for i in range(5):
        fold_tag = 'n{:d}'.format(i)
        fold_files = [
            f for f in files
            if fold_tag in f
        ]
        fold_dict = {}
        for loss_tag in loss_tags:
            loss_files = [
                f for f in fold_files
                if '-{:}.'.format(loss_tag) in f and optim_tag in f and lr_tag in f and rate_tag in f
            ]
            fold_dict[loss_tag] = get_fold_dict(loss_files, path)
        all_dicts.append(fold_dict)
    
    return all_dicts


def get_table_vector(all_dicts, losses, names):
    table_vector = []
    latex_array = ''
    for i, (name, loss) in enumerate(zip(names, losses)):
        loss_table = [name]
        latex_dsc = name
        latex_fn = name
        best_fold = []
        for fold_dict in all_dicts:
            best_fold.append([
                np.max(1 - fold_dict[loss]['all_train_dsc'][:, 1:]),
                np.max(1 - fold_dict[loss]['all_val_dsc'][:, 1:]),
                np.max(1 - fold_dict[loss]['all_test_dsc'][:, 1:]),
                np.max(1 - fold_dict[loss]['all_train_fn'][:, 1:]),
                np.max(1 - fold_dict[loss]['all_val_fn'][:, 1:]),
                np.max(1 - fold_dict[loss]['all_test_fn'][:, 1:])
            ])
            fold_metrics = [
                '{:4.2f} [{:4.2f}, {:4.2f}]'.format(
                    np.mean(1 - fold_dict[loss]['all_train_dsc'][:, 1:]),
                    np.min(1 - fold_dict[loss]['all_train_dsc'][:, 1:]),
                    np.max(1 - fold_dict[loss]['all_train_dsc'][:, 1:]),
                    np.std(1 - fold_dict[loss]['all_train_dsc'][:, 1:])
                ),
                '{:4.2f} [{:4.2f}, {:4.2f}]'.format(
                    np.mean(1 - fold_dict[loss]['all_val_dsc'][:, 1:]),
                    np.min(1 - fold_dict[loss]['all_val_dsc'][:, 1:]),
                    np.max(1 - fold_dict[loss]['all_val_dsc'][:, 1:]),
                    np.std(1 - fold_dict[loss]['all_val_dsc'][:, 1:])
                ),
                '{:4.2f} [{:4.2f}, {:4.2f}]'.format(
                    np.mean(1 - fold_dict[loss]['all_test_dsc'][:, 1:]),
                    np.min(1 - fold_dict[loss]['all_test_dsc'][:, 1:]),
                    np.max(1 - fold_dict[loss]['all_test_dsc'][:, 1:]),
                    np.std(1 - fold_dict[loss]['all_test_dsc'][:, 1:])
                ),
                '{:4.2f} [{:4.2f}, {:4.2f}]'.format(
                    np.mean(1 - fold_dict[loss]['all_train_fn'][:, 1:]),
                    np.min(1 - fold_dict[loss]['all_train_fn'][:, 1:]),
                    np.max(1 - fold_dict[loss]['all_train_fn'][:, 1:]),
                    np.std(1 - fold_dict[loss]['all_train_fn'][:, 1:])
                ),
                '{:4.2f} [{:4.2f}, {:4.2f}]'.format(
                    np.mean(1 - fold_dict[loss]['all_val_fn'][:, 1:]),
                    np.min(1 - fold_dict[loss]['all_val_fn'][:, 1:]),
                    np.max(1 - fold_dict[loss]['all_val_fn'][:, 1:]),
                    np.std(1 - fold_dict[loss]['all_val_fn'][:, 1:])
                ),
                '{:4.2f} [{:4.2f}, {:4.2f}]'.format(
                    np.mean(1 - fold_dict[loss]['all_test_fn'][:, 1:]),
                    np.min(1 - fold_dict[loss]['all_test_fn'][:, 1:]),
                    np.max(1 - fold_dict[loss]['all_test_fn'][:, 1:]),
                    np.std(1 - fold_dict[loss]['all_test_fn'][:, 1:])
                ),
            ]
            loss_table += fold_metrics
            latex_dsc +=  ' & {:} '.format(' & '.join(fold_metrics[:3])).replace('±', '$\pm$')
            latex_fn +=  ' & {:} '.format(' & '.join(fold_metrics[3:])).replace('±', '$\pm$')
        best_values = ['{:4.2f}'.format(val) for val in np.mean(best_fold, axis=0)]
        loss_table += best_values
        latex_array += '{:} & {:} \\\\\n'.format(name, ' & '.join(best_values))
        table_vector.append(loss_table)
    return table_vector, latex_array


def violin_analysis(all_dicts):
    plt.ioff()
    figs = []
    for i, fold_dict in enumerate(all_dicts):        
        plt.figure()
        figs.append(plt.gcf().number)

        ax = plt.subplot(2, 2, 1)
        ax.set_title('Training fold {:d} DSC'.format(i))
        plot_violin([1 - v['all_val_dsc'][:, 1:].flatten() for v in fold_dict.values()], names, ax)
        
        ax = plt.subplot(2, 2, 2)
        ax.set_title('Training fold {:d} TP'.format(i))
        plot_violin([1 - v['all_val_fn'][:, 1:].flatten() for v in fold_dict.values()], names, ax)
              
        ax = plt.subplot(2, 2, 3)
        ax.set_title('Testing fold {:d} DSC'.format(i))
        plot_violin([1 - v['all_test_dsc'][:, 1:].flatten() for v in fold_dict.values()], names, ax)
               
        ax = plt.subplot(2, 2, 4)
        ax.set_title('Testing fold {:d} TP'.format(i))
        plot_violin([1 - v['all_test_fn'][:, 1:].flatten() for v in fold_dict.values()], names, ax)
    
    plt.figure()
    figs.append(plt.gcf().number)
    
    ax = plt.subplot(2, 2, 1)
    ax.set_title('All folds training DSC')
    data = np.concatenate([
        [1 - v['all_val_dsc'][1:,:] for v in fold_dict.values()]
        for fold_dict in all_dicts
    ], axis=1)
    plot_violin(data.flatten(), names, ax)
    
    ax = plt.subplot(2, 2, 2)
    ax.set_title('All folds training TP')
    data = np.concatenate([
        [1 - v['all_val_fn'][1:,:] for v in fold_dict.values()]
        for fold_dict in all_dicts
    ], axis=1)
    plot_violin(data.flatten(), names, ax)
      
    ax = plt.subplot(2, 2, 3)
    ax.set_title('All folds testing DSC')
    data = np.concatenate([
        [1 - v['all_test_dsc'][1:,:] for v in fold_dict.values()]
        for fold_dict in all_dicts
    ], axis=1)
    plot_violin(data.flatten(), names, ax)
       
    ax = plt.subplot(2, 2, 4)
    ax.set_title('All folds testing TP')
    data = np.concatenate([
        [1 - v['all_test_fn'][1:,:] for v in fold_dict.values()]
        for fold_dict in all_dicts
    ], axis=1)
    plot_violin(data.flatten(), names, ax)
    
    plt.show()
    return figs


def save_violin_analysis(path, all_dicts):
    plt.ioff()
    figs = []
    for i, fold_dict in enumerate(all_dicts):                
        save_violin(
            [1 - v['all_val_dsc'][:, 1:].flatten() for v in fold_dict.values()],
            names, os.path.join(path, 'train_violin_dsc_n{:d}'.format(i)),
            title='Training images fold {:d} DSC'.format(i)
        )
        save_violin(
            [1 - v['all_val_fn'][:, 1:].flatten() for v in fold_dict.values()],
            names, os.path.join(path, 'train_violin_tp_n{:d}'.format(i)),
            title='Training images fold {:d} TPf'.format(i)
        )
        save_violin(
            [1 - v['all_test_dsc'][:, 1:].flatten() for v in fold_dict.values()],
            names, os.path.join(path, 'test_violin_dsc_n{:d}'.format(i)),
            title='Testing images fold {:d} DSC'.format(i)
        )
        save_violin(
            [1 - v['all_test_fn'][:, 1:].flatten() for v in fold_dict.values()],
            names, os.path.join(path, 'test_violin_tp_n{:d}'.format(i)),
            title='Testing images fold {:d} TPf'.format(i)
        )
    
    data = np.concatenate([
        [1 - v['all_val_dsc'][:, 1:] for v in fold_dict.values()]
        for fold_dict in all_dicts
    ], axis=1)
    save_violin(
        [datum.flatten() for datum in data],
        names, os.path.join(path, 'train_violin_dsc'),
        title='Training images DSC'
    )
    data = np.concatenate([
        [1 - v['all_val_fn'][:, 1:] for v in fold_dict.values()]
        for fold_dict in all_dicts
    ], axis=1)
    save_violin(
        [datum.flatten() for datum in data],
        names, os.path.join(path, 'train_violin_tp'),
        title='Training images TPf'
    )
    data = np.concatenate([
        [1 - v['all_test_dsc'][:, 1:] for v in fold_dict.values()]
        for fold_dict in all_dicts
    ], axis=1)
    save_violin(
        [datum.flatten() for datum in data],
        names, os.path.join(path, 'test_violin_dsc'),
        title='Testing images DSC'
    )
    data = np.concatenate([
        [1 - v['all_test_fn'][:, 1:] for v in fold_dict.values()]
        for fold_dict in all_dicts
    ], axis=1)
    save_violin(
        [datum.flatten() for datum in data],
        names, os.path.join(path, 'test_violin_tp'),
        title='Testing images TPf'
    )
    
    plt.show()



def band_analysis(all_dicts):
    plt.ioff()
    figs = []
    for i, fold_dict in enumerate(all_dicts):        
        plt.figure()
        figs.append(plt.gcf().number)
               
        ax = plt.subplot(2, 3, 1)
        y_dsc = [1 - v['mean_train_dsc'] for v in fold_dict.values()]
        yinf_dsc = [1 - v['min_train_dsc'] for v in fold_dict.values()]
        ysup_dsc = [1 - v['max_train_dsc'] for v in fold_dict.values()]       
        x = range(len(y_dsc[0]))
        plot_bands(
            x, y_dsc, yinf_dsc, ysup_dsc, title='Training patches DSC metric fold n{:d}'.format(i),
            ymin=0, ymax=1, ylabel='DSC metric', legend=names, ax=ax
        )
        
        ax = plt.subplot(2, 3, 2)
        y_dsc = [1 - v['mean_val_dsc'] for v in fold_dict.values()]
        yinf_dsc = [1 - v['min_val_dsc'] for v in fold_dict.values()]
        ysup_dsc = [1 - v['max_val_dsc'] for v in fold_dict.values()]
        plot_bands(
            x, y_dsc, yinf_dsc, ysup_dsc, title='Training images DSC metric fold n{:d}'.format(i),
            ymin=0, ymax=1, ylabel='DSC metric', legend=names, ax=ax
        )
              
        ax = plt.subplot(2, 3, 3)
        y_dsc = [1 - v['mean_test_dsc'] for v in fold_dict.values()]
        yinf_dsc = [1 - v['min_test_dsc'] for v in fold_dict.values()]
        ysup_dsc = [1 - v['max_test_dsc'] for v in fold_dict.values()]
        plot_bands(
            x, y_dsc, yinf_dsc, ysup_dsc, title='Testing images DSC metric fold n{:d}'.format(i),
            ymin=0, ymax=1, ylabel='DSC metric', legend=names, ax=ax
        )
               
        ax = plt.subplot(2, 3, 4)
        y_dsc = [1 - v['mean_train_fn'] for v in fold_dict.values()]
        yinf_dsc = [1 - v['min_train_fn'] for v in fold_dict.values()]
        ysup_dsc = [1 - v['max_train_fn'] for v in fold_dict.values()]       
        plot_bands(
            x, y_dsc, yinf_dsc, ysup_dsc, title='Training patches TP metric fold n{:d}'.format(i),
            ymin=0, ymax=1, ylabel='TP metric', legend=names, ax=ax
        )    
        
        ax = plt.subplot(2, 3, 5)
        y_dsc = [1 - v['mean_val_fn'] for v in fold_dict.values()]
        yinf_dsc = [1 - v['min_val_fn'] for v in fold_dict.values()]
        ysup_dsc = [1 - v['max_val_fn'] for v in fold_dict.values()]
        plot_bands(
            x, y_dsc, yinf_dsc, ysup_dsc, title='Training images TP metric fold n{:d}'.format(i),
            ymin=0, ymax=1, ylabel='TP metric', legend=names, ax=ax
        )

        ax = plt.subplot(2, 3, 6)
        y_dsc = [1 - v['mean_test_fn'] for v in fold_dict.values()]
        yinf_dsc = [1 - v['min_test_fn'] for v in fold_dict.values()]
        ysup_dsc = [1 - v['max_test_fn'] for v in fold_dict.values()]
        plot_bands(
            x, y_dsc, yinf_dsc, ysup_dsc, title='Testing images TP metric fold n{:d}'.format(i),
            ymin=0, ymax=1, ylabel='TP metric', legend=names, ax=ax
        )

    
    plt.figure()
    figs.append(plt.gcf().number)
    plt.show()
    
    return figs


def save_band_analysis(path, all_dicts):
    plt.ioff()
    for i, fold_dict in enumerate(all_dicts):    
        data = [1 - v['all_train_dsc'] for v in fold_dict.values()]
        save_all_bands(path, data, 'DSC', 'Training patches DSC metric fold n{:d}'.format(i), 'train', fold=i)

        data = [1 - v['all_val_dsc'] for v in fold_dict.values()]
        save_all_bands(path, data, 'DSC', 'Training images DSC metric fold n{:d}'.format(i), 'val', fold=i)

        data = [1 - v['all_test_dsc'] for v in fold_dict.values()]
        save_all_bands(path, data, 'DSC', 'Testing images DSC metric fold n{:d}'.format(i), 'test', fold=i)

        data = [1 - v['all_train_fn'] for v in fold_dict.values()]
        save_all_bands(path, data, 'TP', 'Training patches TPf metric fold n{:d}'.format(i), 'train', fold=i)

        data = [1 - v['all_val_fn'] for v in fold_dict.values()]
        save_all_bands(path, data, 'TP', 'Training images TPf metric fold n{:d}'.format(i), 'val', fold=i)

        data = [1 - v['all_test_fn'] for v in fold_dict.values()]
        save_all_bands(path, data, 'TP', 'Testing images TPf metric fold n{:d}'.format(i), 'test', fold=i)
        
    data = np.concatenate([
        [1 - v['all_train_dsc'] for v in fold_dict.values()]
        for fold_dict in all_dicts
    ], axis=1)
    save_all_bands(path, data, 'DSC', 'Training patches DSC metric', 'train')
  
    data = np.concatenate([
        [1 - v['all_val_dsc'] for v in fold_dict.values()]
        for fold_dict in all_dicts
    ], axis=1)
    save_all_bands(path, data, 'DSC', 'Training images DSC metric', 'val')
    
    data = np.concatenate([
        [1 - v['all_test_dsc'] for v in fold_dict.values()]
        for fold_dict in all_dicts
    ], axis=1)
    save_all_bands(path, data, 'DSC', 'Testing images DSC metric', 'test')
    
    data = np.concatenate([
        [1 - v['all_train_fn'] for v in fold_dict.values()]
        for fold_dict in all_dicts
    ], axis=1)
    save_all_bands(path, data, 'TP', 'Training patches TPf metric', 'train')
    
    data = np.concatenate([
        [1 - v['all_val_fn'] for v in fold_dict.values()]
        for fold_dict in all_dicts
    ], axis=1)
    save_all_bands(path, data, 'TP', 'Training images TPf metric', 'val')
    
    data = np.concatenate([
        [1 - v['all_test_fn'] for v in fold_dict.values()]
        for fold_dict in all_dicts
    ], axis=1)
    save_all_bands(path, data, 'TP', 'Testing images TPf metric', 'test')
    plt.show()

def plot_violin(data, labels, ax, ymin=0, ymax=1):
    plt.violinplot(data)
    ax.get_xaxis().set_tick_params(direction='out')
#     ax.xaxis.set_ticks_position('bottom')
#     ax.set_xticks(np.arange(1, len(labels) + 1))
#     ax.set_xticklabels(labels)
    ax.set_xlim(0.25, len(labels) + 0.75)
    ax.set_ylim(ymin=ymin, ymax=ymax)
    plt.xticks(np.arange(1, len(labels) + 1), labels, rotation=45)
    plt.margins(0.3)
    plt.subplots_adjust(bottom=0.15)
    
    
def save_violin(data, labels, file, title=''):
    plt.ioff()
    fig = plt.figure()
    ax = fig.add_subplot()
    plot_violin(data, names, ax)
    plt.title(title)
    plt.savefig('{:}.png'.format(file), dpi=1200)
    ax.set_rasterized(True)
    plt.savefig('{:}.eps'.format(file))
    plt.close(fig)
    plt.show()

    
def plot_bands(
    x, y, yinf, ysup, ax, xmin=None, xmax=None, ymin=0, ymax=1,
    title='', xlabel='Epoch', ylabel='Metric', legend=None
):
    # Init
    if xmin is None:
        xmin = np.min(x)
    if xmax is None:
        xmax = np.max(x)
    if ymin is None:
        ymin = np.min(yinf)
    if ymax is None:
        ymax = np.max(ysup)

    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

    colomap = ['b', 'g', 'c', 'r', 'm', 'y', 'k']

    if yinf is not None and ysup is not None:
        for yi, yinfi, ysupi, ci in zip(y, yinf, ysup, colomap):
            ax.plot(x, yi, '-', color=ci)
            ax.fill_between(x, yinfi, ysupi, alpha=0.2, color=ci)
    else:
        for yi, ci in zip(y, colomap):
            ax.plot(x, yi, '-', color=ci, linewidth=2.0)

    ax.set_xlim(xmin=xmin, xmax=xmax)
    ax.set_ylim(ymin=ymin, ymax=ymax)

    if legend is not None:
        ax.legend(legend)


def save_all_bands(path, data, metric, title, mode, fold=None):
    y = [np.mean(datum, axis=0) for datum in data]
    yinf = [np.min(datum, axis=0) for datum in data]
    ysup = [np.max(datum, axis=0) for datum in data]
    x = range(len(y[0]))
    
    if fold is None:
        file_bands =  os.path.join(path, 'bands_{:}_{:}'.format(metric.lower(), mode))
        file_sbands =  os.path.join(path, 'sbands_{:}_{:}'.format(metric.lower(), mode))
        file_curves =  os.path.join(path, 'curves_{:}_{:}'.format(metric.lower(), mode))
    else:
        file_bands =  os.path.join(path, 'bands_{:}_{:}_n{:d}'.format(metric.lower(), mode, fold))
        file_sbands =  os.path.join(path, 'sbands_{:}_{:}_n{:d}'.format(metric.lower(), mode, fold))
        file_curves =  os.path.join(path, 'curves_{:}_{:}_n{:d}'.format(metric.lower(), mode, fold))
    
    save_bands(
        x, y, yinf, ysup, title=title, ymin=0, ymax=1,
        ylabel='{:} metric'.format(metric.upper()),
        legend=names,file=file_bands
    )
    yinf = [np.mean(datum, axis=0) - np.std(datum, axis=0) for datum in data]
    ysup = [np.mean(datum, axis=0) + np.std(datum, axis=0) for datum in data]
    save_bands(
        x, y, yinf, ysup, title=title, ymin=0, ymax=1,
        ylabel='{:} metric'.format(metric.upper()),
        legend=names, file=file_sbands
    )
    save_bands(
        x, y, None, None, title=title, ymin=0, ymax=1,
        ylabel='{:} metric'.format(metric.upper()),
        legend=names, file=file_curves
    )

        
        
def save_bands(
    x, y, yinf, ysup, file, xmin=None, xmax=None, ymin=0, ymax=1,
    title='', xlabel='Epoch', ylabel='Metric', legend=None
):
    plt.ioff()
    fig = plt.figure()
    ax = fig.add_subplot()
    plot_bands(x, y, yinf, ysup, ax, xmin, xmax, ymin, ymax, title, xlabel, ylabel, legend)
    plt.savefig('{:}.png'.format(file), dpi=1200)
    ax.set_rasterized(True)
    plt.savefig('{:}.eps'.format(file))
    plt.close(fig)
    plt.show()

## Data loading

In [3]:
loss_tags = ['xent', 'gdsc_b', 'dsc', 'mixed', 'focal_w1', 'focal_w2', 'new']
names = ['xent', 'gdsc', 'dsc', 'mixed', 'focal1', 'focal2', 'new']
optim_tag = 'adam'
lr_tag = '1e-04'
rate_tag = 'nr0'

lit_violins = []
lit_bands = []
wmh_violins = []
wmh_bands = []

panda_header = [
    'Loss',
    'Fold1_PatchTrain_DSC', 'Fold1_Train_DSC', 'Fold1_Test_DSC',
    'Fold1_PatchTrain_FN', 'Fold1_Train_FN', 'Fold1_Test_FN',
    'Fold2_PatchTrain_DSC', 'Fold2_Train_DSC', 'Fold2_Test_DSC',
    'Fold2_PatchTrain_FN', 'Fold2_Train_FN', 'Fold2_Test_FN',
    'Fold3_PatchTrain_DSC', 'Fold3_Train_DSC', 'Fold3_Test_DSC',
    'Fold3_PatchTrain_FN', 'Fold3_Train_FN', 'Fold3_Test_FN',
    'Fold4_PatchTrain_DSC', 'Fold4_Train_DSC', 'Fold4_Test_DSC',
    'Fold4_PatchTrain_FN', 'Fold4_Train_FN', 'Fold4_Test_FN',
    'Fold5_PatchTrain_DSC', 'Fold5_Train_DSC', 'Fold5_Test_DSC',
    'Fold5_PatchTrain_FN', 'Fold5_Train_FN', 'Fold5_Test_FN',
    'Mean_PatchTrain_DSC', 'Mean_Train_DSC', 'Mean_Test_DSC',
    'Mean_PatchTrain_FN', 'Mean_Train_FN', 'Mean_Test_FN',
]

### LIT data

In [4]:
lit_path = '/home/mariano/data/Loss/lit_longitudinal'
lit_dicts = get_all_dicts(lit_path)

In [5]:
lit_vector, lit_latex = get_table_vector(lit_dicts, loss_tags, names)
pd.DataFrame(lit_vector, columns=panda_header)

,Loss,Fold1_PatchTrain_DSC,Fold1_Train_DSC,Fold1_Test_DSC,Fold1_PatchTrain_FN,Fold1_Train_FN,Fold1_Test_FN,Fold2_PatchTrain_DSC,Fold2_Train_DSC,Fold2_Test_DSC,...,Fold5_Test_DSC,Fold5_PatchTrain_FN,Fold5_Train_FN,Fold5_Test_FN,Mean_PatchTrain_DSC,Mean_Train_DSC,Mean_Test_DSC,Mean_PatchTrain_FN,Mean_Train_FN,Mean_Test_FN
0,xent,"0.31 [0.00, 0.68]","0.31 [0.00, 0.52]","0.53 [0.00, 0.65]","0.27 [0.00, 0.64]","0.26 [0.00, 0.52]","0.48 [0.00, 0.75]","0.34 [0.00, 0.69]","0.37 [0.00, 0.57]","0.13 [0.00, 0.38]",...,"0.27 [0.00, 0.53]","0.22 [0.00, 0.55]","0.23 [0.00, 0.62]","0.26 [0.00, 0.62]",0.64,0.51,0.46,0.62,0.55,0.49
1,gdsc,"0.72 [0.15, 0.94]","0.42 [0.20, 0.50]","0.56 [0.34, 0.67]","0.84 [0.18, 0.96]","0.58 [0.26, 0.83]","0.64 [0.27, 0.99]","0.73 [0.20, 0.95]","0.48 [0.12, 0.55]","0.29 [0.07, 0.41]",...,"0.38 [0.25, 0.47]","0.80 [0.17, 0.96]","0.49 [0.29, 0.81]","0.57 [0.38, 0.83]",0.93,0.54,0.53,0.96,0.81,0.77
2,dsc,"0.75 [0.24, 0.96]","0.40 [0.14, 0.50]","0.55 [0.19, 0.67]","0.94 [0.58, 0.99]","0.62 [0.46, 0.90]","0.66 [0.42, 0.99]","0.76 [0.27, 0.98]","0.45 [0.20, 0.57]","0.30 [0.08, 0.44]",...,"0.37 [0.08, 0.47]","0.93 [0.32, 0.99]","0.48 [0.24, 0.85]","0.56 [0.33, 0.80]",0.96,0.55,0.52,0.99,0.89,0.84
3,mixed,"0.69 [0.12, 0.95]","0.41 [0.13, 0.50]","0.57 [0.36, 0.69]","0.83 [0.12, 0.97]","0.61 [0.23, 0.86]","0.69 [0.43, 0.98]","0.69 [0.14, 0.95]","0.46 [0.23, 0.59]","0.34 [0.00, 0.50]",...,"0.35 [0.00, 0.43]","0.80 [0.00, 0.98]","0.57 [0.00, 0.84]","0.61 [0.00, 0.84]",0.93,0.55,0.54,0.98,0.86,0.82
4,focal1,"0.26 [0.00, 0.62]","0.23 [0.00, 0.48]","0.56 [0.00, 0.74]","0.21 [0.00, 0.56]","0.17 [0.00, 0.43]","0.49 [0.00, 0.80]","0.29 [0.00, 0.61]","0.34 [0.00, 0.53]","0.07 [0.00, 0.39]",...,"0.23 [0.00, 0.50]","0.19 [0.00, 0.51]","0.17 [0.00, 0.55]","0.20 [0.00, 0.68]",0.59,0.48,0.46,0.52,0.49,0.49
5,focal2,"0.64 [0.14, 0.90]","0.45 [0.19, 0.59]","0.69 [0.47, 0.78]","0.68 [0.14, 0.98]","0.43 [0.23, 0.62]","0.75 [0.33, 0.92]","0.66 [0.15, 0.89]","0.49 [0.29, 0.61]","0.30 [0.00, 0.54]",...,"0.40 [0.00, 0.58]","0.67 [0.09, 0.96]","0.44 [0.09, 0.66]","0.50 [0.00, 0.71]",0.89,0.60,0.56,0.97,0.70,0.63
6,new,"0.54 [0.17, 0.85]","0.39 [0.04, 0.57]","0.60 [0.04, 0.77]","0.99 [0.62, 1.00]","0.74 [0.51, 0.95]","0.92 [0.74, 1.00]","0.56 [0.22, 0.76]","0.43 [0.11, 0.55]","0.39 [0.09, 0.58]",...,"0.36 [0.02, 0.46]","0.99 [0.82, 1.00]","0.72 [0.49, 0.93]","0.72 [0.57, 0.88]",0.79,0.58,0.56,1.00,0.94,0.90


In [6]:
print(lit_latex)

xent & 0.64 & 0.51 & 0.46 & 0.62 & 0.55 & 0.49 \\
gdsc & 0.93 & 0.54 & 0.53 & 0.96 & 0.81 & 0.77 \\
dsc & 0.96 & 0.55 & 0.52 & 0.99 & 0.89 & 0.84 \\
mixed & 0.93 & 0.55 & 0.54 & 0.98 & 0.86 & 0.82 \\
focal1 & 0.59 & 0.48 & 0.46 & 0.52 & 0.49 & 0.49 \\
focal2 & 0.89 & 0.60 & 0.56 & 0.97 & 0.70 & 0.63 \\
new & 0.79 & 0.58 & 0.56 & 1.00 & 0.94 & 0.90 \\



In [7]:
# lit_violins = violin_analysis(lit_dicts)
save_violin_analysis(lit_path, lit_dicts)

In [8]:
# lit_bands = band_analysis(lit_path, lit_dicts)
save_band_analysis(lit_path, lit_dicts)

### WMH data

In [9]:
wmh_path = '/home/mariano/data/Loss/wmh'
wmh_dicts = get_all_dicts(wmh_path)

In [10]:
wmh_vector, wmh_latex = get_table_vector(wmh_dicts, loss_tags, names)
pd.DataFrame(wmh_vector, columns=panda_header)

,Loss,Fold1_PatchTrain_DSC,Fold1_Train_DSC,Fold1_Test_DSC,Fold1_PatchTrain_FN,Fold1_Train_FN,Fold1_Test_FN,Fold2_PatchTrain_DSC,Fold2_Train_DSC,Fold2_Test_DSC,...,Fold5_Test_DSC,Fold5_PatchTrain_FN,Fold5_Train_FN,Fold5_Test_FN,Mean_PatchTrain_DSC,Mean_Train_DSC,Mean_Test_DSC,Mean_PatchTrain_FN,Mean_Train_FN,Mean_Test_FN
0,xent,"0.85 [0.45, 0.97]","0.69 [0.52, 0.76]","0.67 [0.53, 0.71]","0.84 [0.39, 0.99]","0.65 [0.42, 0.73]","0.63 [0.43, 0.71]","0.85 [0.43, 0.97]","0.69 [0.52, 0.73]","0.69 [0.50, 0.74]",...,"0.56 [0.35, 0.64]","0.85 [0.42, 0.98]","0.66 [0.41, 0.74]","0.55 [0.26, 0.64]",0.97,0.74,0.72,0.99,0.72,0.71
1,gdsc,"0.82 [0.61, 0.88]","0.69 [0.51, 0.74]","0.66 [0.49, 0.70]","0.81 [0.56, 0.88]","0.62 [0.41, 0.72]","0.59 [0.39, 0.70]","0.81 [0.63, 0.88]","0.68 [0.55, 0.74]","0.69 [0.54, 0.74]",...,"0.55 [0.45, 0.64]","0.81 [0.58, 0.88]","0.64 [0.46, 0.75]","0.54 [0.35, 0.65]",0.88,0.73,0.72,0.88,0.70,0.70
2,dsc,"0.83 [0.67, 0.89]","0.67 [0.50, 0.72]","0.63 [0.46, 0.69]","0.85 [0.72, 0.91]","0.62 [0.52, 0.71]","0.58 [0.47, 0.69]","0.83 [0.66, 0.89]","0.68 [0.50, 0.73]","0.68 [0.50, 0.72]",...,"0.55 [0.34, 0.67]","0.84 [0.73, 0.90]","0.64 [0.57, 0.73]","0.53 [0.45, 0.64]",0.89,0.73,0.72,0.90,0.70,0.70
3,mixed,"0.84 [0.65, 0.91]","0.69 [0.60, 0.74]","0.65 [0.57, 0.70]","0.85 [0.70, 0.92]","0.62 [0.55, 0.72]","0.58 [0.51, 0.69]","0.84 [0.65, 0.91]","0.70 [0.60, 0.75]","0.70 [0.57, 0.74]",...,"0.58 [0.46, 0.68]","0.85 [0.70, 0.91]","0.64 [0.57, 0.75]","0.53 [0.47, 0.66]",0.91,0.75,0.73,0.92,0.72,0.71
4,focal1,"0.78 [0.22, 0.96]","0.67 [0.27, 0.74]","0.65 [0.20, 0.70]","0.73 [0.16, 0.95]","0.59 [0.17, 0.70]","0.57 [0.12, 0.68]","0.78 [0.40, 0.96]","0.67 [0.48, 0.72]","0.68 [0.48, 0.73]",...,"0.55 [0.20, 0.63]","0.74 [0.25, 0.95]","0.61 [0.23, 0.71]","0.49 [0.14, 0.62]",0.96,0.73,0.71,0.96,0.69,0.68
5,focal2,"0.87 [0.62, 0.97]","0.73 [0.66, 0.78]","0.70 [0.66, 0.73]","0.92 [0.63, 1.00]","0.71 [0.57, 0.80]","0.69 [0.58, 0.78]","0.87 [0.61, 0.97]","0.73 [0.65, 0.77]","0.72 [0.64, 0.76]",...,"0.61 [0.56, 0.70]","0.93 [0.68, 1.00]","0.73 [0.65, 0.82]","0.63 [0.54, 0.73]",0.97,0.77,0.76,0.99,0.81,0.79
6,new,"0.70 [0.42, 0.89]","0.70 [0.55, 0.75]","0.65 [0.48, 0.71]","0.99 [0.91, 1.00]","0.84 [0.75, 0.95]","0.82 [0.71, 0.96]","0.69 [0.36, 0.87]","0.69 [0.48, 0.75]","0.67 [0.47, 0.73]",...,"0.58 [0.40, 0.66]","0.99 [0.93, 1.00]","0.86 [0.76, 0.96]","0.79 [0.67, 0.94]",0.87,0.75,0.73,1.00,0.97,0.96


In [11]:
print(wmh_latex)

xent & 0.97 & 0.74 & 0.72 & 0.99 & 0.72 & 0.71 \\
gdsc & 0.88 & 0.73 & 0.72 & 0.88 & 0.70 & 0.70 \\
dsc & 0.89 & 0.73 & 0.72 & 0.90 & 0.70 & 0.70 \\
mixed & 0.91 & 0.75 & 0.73 & 0.92 & 0.72 & 0.71 \\
focal1 & 0.96 & 0.73 & 0.71 & 0.96 & 0.69 & 0.68 \\
focal2 & 0.97 & 0.77 & 0.76 & 0.99 & 0.81 & 0.79 \\
new & 0.87 & 0.75 & 0.73 & 1.00 & 0.97 & 0.96 \\



In [12]:
# wmh_violins = violin_analysis(wmh_path, wmh_dicts)
save_violin_analysis(wmh_path, wmh_dicts)

In [13]:
# wmh_bands = band_analysis(wmh_dicts)
save_band_analysis(wmh_path, wmh_dicts)